# Annotate Dataset and Train spaCy NER Model

Read and Parse the CSV Data

In [2]:
import csv

In [3]:
# Read the CSV file
with open('event_data.csv', mode='r') as file:
    reader = csv.reader(file)
    # Skip the header
    next(reader)
    rows = [row for row in reader]


Create Annotated Data

In [5]:
annotated_data = []

for row in rows:
    verb, date, time, country = row
    sentence = f"{verb} an event on {date} at {time} in {country}"
    start_date = sentence.find(date)
    end_date = start_date + len(date)
    start_time = sentence.find(time)
    end_time = start_time + len(time)
    
    annotated_data.append((sentence, {"entities": [(start_date, end_date, "DATE"), (start_time, end_time, "TIME")]}))


In [10]:
# !pip install --upgrade setuptools
!pip install spacy
!pip install -U spacy[transformers]
!pip install sklearn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [8]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding

Convert to spaCy Format

In [11]:
nlp = spacy.blank("en")
examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in annotated_data]
# print(examples)

C:\Users\anind\AppData\Roaming\Python\Python311\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\anind\AppData\Roaming\Python\Python311\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


ValueError: [E103] Trying to set conflicting doc.ents: '(21, 35, 'DATE')' and '(26, 35, 'TIME')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.

Train the spaCy Model

In [15]:
import random
ner1 = nlp.add_pipe("ner", last=True)
ner1.add_label("DATE")
ner1.add_label("TIME")
optimizer = nlp.begin_training()

for itn in range(20):  # Number of iterations
    random.shuffle(examples)
    losses = {}
    batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        nlp.update(batch, drop=0.5, losses=losses)
    print(f"Iteration {itn}, Loss: {losses}")


ValueError: [E007] 'ner' already exists in pipeline. Existing names: ['ner']

In [ ]:
nlp.to_disk("event_date_time_model")